In [ ]:
import string
import re
import pandas as pd

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def count_remover(text,threshold=4):
    if len(text.split())<threshold:
        return pd.NaT
    else:
        return text

data=pd.read_json('./data/tweets.json' ,lines=True)  #download this file from: https://nuage.lip6.fr/s/XKkcWLAiDiykZ4D
df=data[["Text","CreatedAt"]].rename(columns={"CreatedAt":"time"})
df['abstract'] = df['Text'].str.replace(r'@\w+', '')
df['abstract'] = df['abstract'].apply(lambda x: re.sub(r"http\S+", "", x))
df['abstract'] = df['abstract'].apply(lambda x: remove_punct(x))
df['abstract'] = df['abstract'].apply(lambda x: count_remover(x))
df=df.dropna()
df['year'] = pd.to_datetime(df['time'], infer_datetime_format=True)
df['year'] = df['year'].dt.to_period('M')
df['year'] = df['year'].apply(lambda x: x.ordinal)

df=df.sort_values("year")
df=df.dropna()
df=df.reset_index(drop=True)
df=df[["abstract","year"]]
df=df.reset_index()

In [ ]:
from models.antm import ANTM
window_size=6
overlap=2

#take a random sample for example
dt=df.sample(n = 5000)
dt=dt.sort_values("year")
dt=dt.reset_index(drop=True)
dt=dt.reset_index()

evolving_topics=ANTM(dt,overlap,window_size,mode="data2vec",num_words=10)